# RAG(검색-증강 생성)
`Retrieval-Augmented Generation`
- LLM이 외부 데이터를 컨텍스트로 활용

### 사전처리 단계
1. 문서 불러오기 (Loading)
2. 텍스트 나누기 (Splitting)
3. 숫자로 바꾸기 (Embedding)
4. 저장하기 (VectorStore)

In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
# 1번 2번 과정 생략

from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [6]:
%pip install -q ' langchain_chroma>=0.1.2' langchain_community faiss-cpu


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
# 3번 과정
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# OpenAI 임베딩 모델
embedding = OpenAIEmbeddings()

# Chroma()

x = embedding.embed_query(documents[0].page_content)
len(x) # 1536: OpenAI 임베딩 컬럼 갯수

# Memory에 저장되는 임시 메모리.
vectorstore = Chroma.from_documents(
    documents,
    embedding=embedding
, )

In [11]:
vectorstore.similarity_search(
        '독수리 부리는 왜 노란 색일까?', k=2
    )
# 언어가 달라도 임베딩된 위치가 비슷함.

[Document(id='ac6ded99-a016-4187-87f5-c2975c592f26', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='e99d11af-887f-414e-996d-e5c02bdf6cc2', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [12]:
%pip install -q pymupdf


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [23]:
# 1. Load
from langchain_community.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader('./spri.pdf')
docs = loader.load()

print('원본 pdf 장수: ', len(docs))

원본 pdf 장수:  23


In [22]:
# 2. Split
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 500글자 당 1 청크 / 50글자는 겹치게 나눈다
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)

# for doc in split_docs:
#     print('\n')
#     print(doc)

print('분할 후 chunk 수: ', len(split_docs)) # 72

분할 후 chunk 수:  72


In [24]:
# 3. 벡터 임베딩 + 4. 벤터스토어 저장

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=split_docs, embedding=embedding)

# Test
vectorstore.similarity_search('미국 대통령 관련 문서 가져오기', k=4)

[Document(id='c6f18cc9-eaad-4d7a-abe6-801a457524c4', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': './spri.pdf', 'file_path': './spri.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 3}, page_content='1. 정책/법제  \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표 \nn 미국 바이든 대통령이 ‘안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령’에 서명하고 \n광범위한 행정 조치를 명시\nn 행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 \n보호 △노동자 지원 △혁신과 경쟁 촉진 △국제협력을 골자로 함\nKEY Contents\n£ 바이든 대통령, AI 행정명령 통해 안전하고 신뢰할 수 있는 AI 개발과 활용 추진\nn 미국 바이든 대통령이 2023년 10월 30일 연방정부 차원에서 안전하고 신뢰할 수 있는 AI 개발과 \n사용을 보장하기 위한 행정명령을 발표\n∙행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 보호 \n△노동자 지원 △혁신과 경쟁 촉진 △국제협력에 관한 내용을 포괄'),
 Document(id='3

In [26]:
# 5. 최종 답변
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

# 프롬프트 세팅
prompt = hub.pull('rlm/rag-prompt')

# LLM 모델
llm = ChatOpenAI(model='gpt-4.1-nano')

# 검색기 생성(retriever 생성)
retriever = vectorstore.as_retriever()

chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt | llm | StrOutputParser()
)

chain.invoke('삼성전자 관련 소식 다 가져와')

"삼성전자는 2023년 11월에 '삼성 가우스'라는 자체 개발 생성 AI 모델을 공개했으며, 언어, 코드, 이미지의 3개 모델로 구성되어 있습니다. 이 AI는 온디바이스에서 작동하며, 사용자의 정보 유출 우려없이 안전하게 학습된 것이 특징입니다. 삼성은 앞으로 다양한 제품에 삼성 가우스를 단계적으로 탑재할 계획입니다."

In [37]:
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain_tavily import TavilySearch
from datetime import datetime
# RAG 관련
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool

today = datetime.today().strftime('%D')

llm = ChatOpenAI(model='gpt-4.1-nano')

search_tool = TavilySearch(
    max_results=5,
    topic='general'
)
loader = PyMuPDFLoader('./spri.pdf')
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(docs)
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embedding=embedding)
retriever = vectorstore.as_retriever()

rag_tool = create_retriever_tool(
    retriever,
    name='pdf_search',
    description='PDF 문서에서 질문과 관련된 내용을 검색합니다. ' # Agent가 언제 툴을 쓸지 알게 됨.
)
text = f"""
너는 웹 검색이 가능하고, 2023년 12월 인공지능 산업 최신동향 정보를 담은 pdf 를 검색할 수 있는 어시스턴트야.

- 사용자가 PDF 문서와 관련된 질문(ex. '이 pdf에서', '문서내용', '파일에서)을 하면 반드시 'pdf_search' 도구를 써야해
- 사용자 질문이 팩트체크를 필요로 하고, 최신성이 필요하다 판단되면 web_search를 실행해야해
- 사용자가 일반적인 질문을 하고, 최신성이나 팩트체크가 필요없으면 그냥 답변해
- 뭔가 확실하지 않으면 pdf_search 와 web_search를 모두 실행해서 답변을 생성해

오늘은 {today} 야.
"""

prompt = ChatPromptTemplate.from_messages([
    ('system', text),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')  # 도구(검색) 호출때 필요함
])

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history'
)

agent = create_openai_tools_agent(
    llm=llm,
    tools=[search_tool, rag_tool],
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    memory = memory,
    tools=[search_tool, rag_tool],
    verbose=True)

# agent_executor.invoke({'messages': [{'role': 'human', 'content': '오늘 날씨에 맞는 음악 추천해줘'}]})

In [38]:
agent_executor.invoke({'input': 'pdf에서 삼성전자 관련된 내용들 가져와'})



> Entering new AgentExecutor chain...

Invoking: `pdf_search` with `{'query': '삼성전자'}`


저해상도 이미지의 고해상도 전환도 지원
n IT 전문지 테크리퍼블릭(TechRepublic)은 온디바이스 AI가 주요 기술 트렌드로 부상했다며, 
2024년부터 가우스를 탑재한 삼성 스마트폰이 메타의 라마(Llama)2를 탑재한 퀄컴 기기 및 구글 
어시스턴트를 적용한 구글 픽셀(Pixel)과 경쟁할 것으로 예상
☞ 출처 : 삼성전자, ‘삼성 AI 포럼’서 자체 개발 생성형 AI ‘삼성 가우스’ 공개, 2023.11.08.
삼성전자, ‘삼성 개발자 콘퍼런스 코리아 2023’ 개최, 2023.11.14.
TechRepublic, Samsung Gauss: Samsung Research Reveals Generative AI, 2023.11.08.

SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며,

∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며,

{'input': 'pdf에서 삼성전자 관련된 내용들 가져와',
 'chat_history': [HumanMessage(content='pdf에서 삼성전자 관련된 내용들 가져와', additional_kwargs={}, response_metadata={}),
  AIMessage(content="삼성전자는 최근 '삼성 AI 포럼 2023'에서 자체 개발한 생성형 AI '삼성 가우스'를 공개하였으며, 다양한 제품에 단계적으로 도입할 계획을 발표하였습니다. 삼성 가우스는 온디바이스에서 작동하며, 언어, 코드, 이미지 3가지 모델로 구성되어 있어 외부로 사용자 정보 유출 위험이 없다는 장점이 있습니다. 이 모델은 텍스트 생성, AI 코딩 어시스턴트, 이미지 생성 및 편집 등 다양한 기능을 지원하며, 각기 다른 최적화된 크기의 모델로 구성되어 있습니다.", additional_kwargs={}, response_metadata={})],
 'output': "삼성전자는 최근 '삼성 AI 포럼 2023'에서 자체 개발한 생성형 AI '삼성 가우스'를 공개하였으며, 다양한 제품에 단계적으로 도입할 계획을 발표하였습니다. 삼성 가우스는 온디바이스에서 작동하며, 언어, 코드, 이미지 3가지 모델로 구성되어 있어 외부로 사용자 정보 유출 위험이 없다는 장점이 있습니다. 이 모델은 텍스트 생성, AI 코딩 어시스턴트, 이미지 생성 및 편집 등 다양한 기능을 지원하며, 각기 다른 최적화된 크기의 모델로 구성되어 있습니다."}